### 12) 모형결합(배깅)

In [ ]:
#배깅: 같은 모형을 사용하지만 같은 데이터 샘플을 중복사용하여 서로 다른 결과를 출력하는 다수의 모형을 사용하는 방법

In [4]:
# csv 파일을 로딩

import pandas as pd

df = pd.read_csv("c:/workspace3/data/customer.csv")
df.head()

,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [5]:
df["Churn"].value_counts()

Churn
0    2850
1     483
Name: count, dtype: int64

In [6]:
train_cols = df.columns[0:16]

X = df[train_cols] # 독립변수
y = df["Churn"]

print(type(X), type(y))

print(X.head())
print(y.head())

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
   Int'l Plan  VMail Plan  VMail Message  Day Mins  Day Calls  Day Charge  \
0           0           1             25     265.1        110       45.07   
1           0           1             26     161.6        123       27.47   
2           0           0              0     243.4        114       41.38   
3           1           0              0     299.4         71       50.90   
4           1           0              0     166.7        113       28.34   

   Eve Mins  Eve Calls  Eve Charge  Night Mins  Night Calls  Night Charge  \
0     197.4         99       16.78       244.7           91         11.01   
1     195.5        103       16.62       254.4          103         11.45   
2     121.2        110       10.30       162.6          104          7.32   
3      61.9         88        5.26       196.9           89          8.86   
4     148.3        122       12.61       186.9          121          8.41   



In [7]:
#언더샘플링

from imblearn.under_sampling import RandomUnderSampler

X_sample, y_sample = RandomUnderSampler(random_state=0).fit_resample(X, y)

X_samp = pd.DataFrame(data=X_sample,columns=train_cols )
y_samp = pd.DataFrame(data=y_sample,columns=['Churn'])

df_samp=pd.concat([X_samp,y_samp],axis=1)
df_samp["Churn"].value_counts()

Churn
0    483
1    483
Name: count, dtype: int64

In [8]:
X = df_samp[train_cols] # 독립변수
y = df_samp["Churn"]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [10]:
# BaggingClassifier : 배깅 모형 결합을 위한 클래스
#     base_estimator: 기본 모형
#     n_estimators: 모형 개수. 디폴트 10
#     bootstrap: 데이터의 중복 사용 여부. 디폴트 True
#     max_samples: 데이터 샘플 중 선택할 샘플의 수 혹은 비율. 디폴트 1.0
#     bootstrap_features: 특징 차원의 중복 사용 여부. 디폴트 False
#     max_features: 다차원 독립 변수 중 선택할 차원의 수 혹은 비율 1.0

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

model1=DecisionTreeClassifier(random_state=0)
model2=SVC(kernel='linear')
#                   linear  선형
#                   rbf     비선형
model3=BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, random_state=0)
#                           단일모형                    여러개
model4=BaggingClassifier(KNeighborsClassifier(n_neighbors=2), n_estimators=10, random_state=0)
model5=BaggingClassifier(SVC(), n_estimators=10, random_state=0)

for model in (model1, model2, model3, model4, model5):
    print(model)

    model.fit(X_train,y_train)

    print(model.score(X_train,y_train))
    print(model.score(X_test,y_test))
    print()

DecisionTreeClassifier(random_state=0)
1.0
0.845360824742268

SVC(kernel='linear')
0.7707253886010362
0.7422680412371134

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100,
                  random_state=0)
1.0
0.8917525773195877

BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=2), random_state=0)
0.8963730569948186
0.6288659793814433

BaggingClassifier(estimator=SVC(), random_state=0)
0.6904145077720207
0.6701030927835051

